In [1]:
# Import Modules
import torch
import torch.nn as nn

In [2]:
''' A simple Convolution, Batch Normalization, and Activation Class'''

class ConvBnAct(nn.Module):

    def __init__(self, 
                 n_in,
                 n_out, 
                 kernel_size = 3, 
                 stride = 1, 
                 padding = 0, 
                 groups = 1, 
                 bn = True, 
                 act = True, 
                 bias= False):

        super(ConvBnAct, self).__init__()

        self.conv = nn.Conv2d(in_channels=n_in, 
                              out_channels=n_out, 
                              kernel_size=kernel_size, 
                              stride=stride, 
                              padding=padding, 
                              groups=groups, 
                              bias=bias
                             )
        self.batch_norm = nn.BatchNorm2d(n_out) if bn else nn.Identity()
        self.activation = nn.LeakyReLU() if act else nn.Identity()

    def forward(self, x):

        return self.activation(self.batch_norm(self.conv(x)))
          

In [3]:
x = torch.randn(1,3,224,224)
model = ConvBnAct(3,12)
model(x).shape

torch.Size([1, 12, 222, 222])

In [29]:
class DarkMaskedPRNBlock(nn.Module):
    def __init__(self, 
                 n_in,
                 sparsity =0.5
                 ):

        super(DarkMaskedPRNBlock, self).__init__()

        reduced_channels = int(n_in/2)

        self.conv1 = ConvBnAct(n_in, 
                              reduced_channels, 
                              kernel_size=1, 
                              stride=1, 
                              padding=0, 
                              groups=1, 
                              bn = True,
                              act = True
                             )
        self.conv2 = ConvBnAct(reduced_channels, 
                              n_in,
                              kernel_size=3, 
                              stride=1, 
                              padding=1, 
                              groups=1, 
                              bn = True,
                              act = True
                             )
        self.residual_size = int(n_in * sparsity)

    def forward(self, x):
        identity = x[:, 0:self.residual_size, :, :] # The Portion of features to add
        #print(out.shape)
        out = self.conv2(self.conv1(x))
        out[:, 0:self.residual_size, :, :] += identity

        return out
                        
        
        

In [30]:
# Test
block = DarkMaskedPRNBlock(n_in = 64)
x = torch.randn(1, 64, 112, 112)
block(x).shape

torch.Size([1, 64, 112, 112])

In [41]:
class DarkAsymmetricPRNBlock(nn.Module):
    def __init__(self, 
                 n_in,
                 sparsity =0.5
                 ):

        super(DarkAsymmetricPRNBlock, self).__init__()

        reduced_channels = int(n_in/2)

        self.conv1 = ConvBnAct(n_in, 
                              reduced_channels, 
                              kernel_size=1, 
                              stride=1, 
                              padding=0, 
                              groups=1, 
                              bn = True,
                              act = True
                             )
        self.conv2 = ConvBnAct(reduced_channels, 
                              reduced_channels,
                              kernel_size=3, 
                              stride=1, 
                              padding=1, 
                              groups=1, 
                              bn = True,
                              act = True
                             )
        self.residual_size = int(n_in * sparsity)

    def forward(self, x):
        identity = x[:, 0:self.residual_size, :, :] # The Portion of features to add
        #print(out.shape)
        out = self.conv2(self.conv1(x))
        out += identity

        return out

In [42]:
# Test
block = DarkAsymmetricPRNBlock(n_in = 64)
x = torch.randn(1, 64, 112, 112)
block(x).shape

torch.Size([1, 32, 112, 112])

In [44]:
class DarkResidualBlock(nn.Module):
    def __init__(self, 
                 n_in,
                 sparsity =0.5
                 ):

        super(DarkResidualBlock, self).__init__()

        reduced_channels = int(n_in/2)

        self.conv1 = ConvBnAct(n_in, 
                              reduced_channels, 
                              kernel_size=1, 
                              stride=1, 
                              padding=0, 
                              groups=1, 
                              bn = True,
                              act = True
                             )
        self.conv2 = ConvBnAct(reduced_channels, 
                              n_in,
                              kernel_size=3, 
                              stride=1, 
                              padding=1, 
                              groups=1, 
                              bn = True,
                              act = True
                             )
        self.residual_size = int(n_in * sparsity)

    def forward(self, x):
        identity = x # The Portion of features to add
        #print(out.shape)
        out = self.conv2(self.conv1(x))
        out += identity

        return out

In [45]:
block = DarkResidualBlock(n_in = 64)
x = torch.randn(1, 64, 112, 112)
block(x).shape

torch.Size([1, 64, 112, 112])

In [96]:
class CSPStage(nn.Module):

    def __init__(self,
                 n_in,
                 n_out,
                 num_blocks = 1
                ):

        super(CSPStage, self).__init__()

        compression = 2 if num_blocks > 1 else 1
        
        self.base_layer = nn.Sequential(
            ConvBnAct(n_in, 
                      n_out, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1, 
                      groups=1, 
                      bn=True, 
                      act=True
                     ), 
            ConvBnAct(n_out, 
                      2 * n_out // compression, 
                      kernel_size=1, 
                      stride=1, 
                      padding=0, 
                      groups=1, 
                      bn=True, 
                      act=True
                     ),
        )

        self.dense_block = nn.Sequential(*[DarkResidualBlock(n_out // compression) for _ in range(num_blocks)])

        self.transition_first = ConvBnAct(n_out // compression, 
                                          n_out // compression, 
                                          kernel_size=1, 
                                          stride=1, 
                                          padding=0, 
                                          groups=1, 
                                          bn=True, 
                                          act=True
                                         )
        
        self.transition_last = ConvBnAct(2 * n_out // compression, 
                                         n_out, 
                                         kernel_size=1, 
                                         stride=1, 
                                         padding=0, 
                                         groups=1, 
                                         bn=True, 
                                         act=True
                                        )
    def forward(self, x):
    
        # Process the Input through base layer
        x = self.base_layer(x)
        print(x.shape)
        x1, x2 = x.chunk(2, dim=1)
        print(x1.shape, x2.shape)
        x2 = self.dense_block(x2)
        x2 = self.transition_first(x2)
        print(x2.shape)

        out = torch.cat([x1, x2], dim=1)

        out = self.transition_last(out)

        return out
        
        

In [97]:
# Test
block = CSPStage(n_in = 64, n_out=32, num_blocks=3)
x = torch.randn(1, 64, 112, 112)
block(x).shape

torch.Size([1, 32, 112, 112])
torch.Size([1, 16, 112, 112]) torch.Size([1, 16, 112, 112])
torch.Size([1, 16, 112, 112])


torch.Size([1, 32, 112, 112])

In [6]:
class OSAModule(nn.Module):
    
    def __init__(self, 
                 n_in, # Input Channels
                 n_stage,# Channels in stage
                 n_out, # Output channel (after aggregation)
                 n_layers_per_block # number of layers per block
                ):
        super(OSAModule, self).__init__()
        
        self.layers = []
        n_input = n_in
        
        for _ in range(n_layers_per_block):
            self.layers.append(ConvBnAct(n_input, 
                                         n_stage, 
                                         kernel_size=3, 
                                         stride=1, 
                                         padding=1, 
                                         groups=1, 
                                         bn=True, 
                                         act=True
                                        )
                              ) 
            n_input = n_stage
        self.layers = nn.Sequential(*self.layers)
            
        # Feature Aggregation
        n_input = n_in + (n_layers_per_block * n_stage)
        
        self.feat_aggregation = ConvBnAct(n_input, 
                                          n_out, 
                                          kernel_size=1, 
                                          stride=1, 
                                          padding=0, 
                                          groups=1, 
                                          bn=True, 
                                          act=True
                                         )
    def forward(self, x):
        
        output = []
        output.append(x)
        
        for layer in self.layers:
            x = layer(x)
            output.append(x)
        
        # Concat and Aggregate
        x = torch.cat(output, dim=1)
        x = self.feat_aggregation(x)
        
        return x
        

In [7]:
# Test
block = OSAModule(n_in = 128, #VoVNet-27-Slim-Configuration (Stage-2 OSA Module)
                  n_stage=64, 
                  n_out=128,
                  n_layers_per_block = 5
                 )
x = torch.randn(1, 128, 64, 64)
block(x).shape

torch.Size([1, 128, 64, 64])

In [13]:
class ELAN(nn.Module):
    '''
      Combine CSPNet + VoVNet (OSA Stages)
    '''
    def __init__(self, 
                 n_in, # Input Channels
                 n_stage,# Channels in stage
                 n_out, # Output channel (after aggregation)
                 n_layers_per_block = 2, # number of layers per block
                 n_stages_to_combine = 2 # number of stages before aggregatin
                ):
        super(ELAN, self).__init__()
        self.base_layer = nn.Sequential(
            ConvBnAct(n_in, 
                      2 * n_in, 
                      kernel_size=3, 
                      stride=1, 
                      padding=1, 
                      groups=1, 
                      bn=True, 
                      act=True
                     ), 
            ConvBnAct(2 * n_in, 
                      n_in, 
                      kernel_size=1, 
                      stride=1, 
                      padding=0, 
                      groups=1, 
                      bn=True, 
                      act=True
                     ),
        )
        
        
         # VoVNet layers
        n_input = n_in//2
        self.stage_layers = []
        
        for _ in range(n_stages_to_combine):
            block_layers = []
            for _ in range(n_layers_per_block):
                block_layers.append(ConvBnAct(n_input, 
                                              n_stage, 
                                              kernel_size=3, 
                                              stride=1, 
                                              padding=1, 
                                              groups=1, 
                                              bn=True, 
                                              act=True
                                             )
                                   ) 
                n_input = n_stage
            
            # Collect feature after every stage
            block_layers = nn.Sequential(*block_layers)
            self.stage_layers.append(block_layers)
        
        self.stage_layers = nn.Sequential(*self.stage_layers)
            
        # Feature Aggregation
        n_input = n_in + n_in//2 + (n_stages_to_combine * n_stage)
        
        self.feat_aggregation = ConvBnAct(n_input, 
                                          n_out, 
                                          kernel_size=1, 
                                          stride=1, 
                                          padding=0, 
                                          groups=1, 
                                          bn=True, 
                                          act=True
                                         )
    def forward(self, x):
        output = []
        output.append(x)
        
        # Process the Input through base layer
        x = self.base_layer(x)
        # print(x.shape)

        # Partition in 2 parts
        x1, x2 = x.chunk(2, dim=1)
        output.append(x1)
        
        
        for layer in self.stage_layers:
            x2 = layer(x2)
            output.append(x2)
        
        # Concat and Aggregate
        x = torch.cat(output, dim=1)
        x = self.feat_aggregation(x)
        
        return x
        

In [20]:
# Test
block = ELAN(n_in = 128,
             n_stage=64, 
             n_out=128,
             n_layers_per_block = 2,
             n_stages_to_combine = 2
            )

x = torch.randn(1, 128, 224,224)
block(x).shape

torch.Size([1, 128, 224, 224])

In [19]:
x

tensor([[[[-0.2363,  0.6134, -0.8807,  ...,  0.4019, -0.1975, -1.4138],
          [-1.3937,  0.7445, -0.0719,  ...,  0.6004, -1.0604, -0.2311],
          [-0.7755,  1.0107, -0.2737,  ...,  0.9264,  0.1675, -0.2398],
          ...,
          [-0.0331, -0.4251, -0.8724,  ...,  0.0312,  1.6985,  1.1055],
          [ 0.2811,  0.3370, -2.5318,  ...,  1.8147, -0.0398, -0.2592],
          [-1.3357,  0.5877,  0.5032,  ...,  1.1442,  1.7555, -1.5329]],

         [[ 0.1918, -1.2404, -1.4473,  ..., -1.4050,  0.6037, -0.1062],
          [ 0.7580,  1.2730, -0.7554,  ...,  0.7185,  1.5603, -1.0550],
          [ 1.7376,  0.4641, -0.0997,  ...,  0.2212, -0.9660, -1.1399],
          ...,
          [ 0.8494, -1.0326, -1.5544,  ...,  1.1887,  1.1353,  0.0314],
          [ 0.0111,  0.8195, -0.5983,  ...,  0.5770,  0.3134, -0.1779],
          [ 1.3251,  0.1026, -0.6938,  ..., -0.3294, -0.3461,  0.3597]],

         [[-0.8859, -0.1814,  0.9488,  ..., -0.4712,  0.1985, -0.4933],
          [ 1.1694, -2.5618,  

[5, 7, 9]

88

In [21]:
import torch
import torch.nn as nn


class GELANBlock(nn.Module):
    """
    Generalized Efficient Layer Aggregation Network (GELAN) Block.
    Supports any computational block for flexible configurations.
    """
    def __init__(self, in_channels, out_channels, block_type="CSP", depth=2):
        super(GELANBlock, self).__init__()
        self.depth = depth
        self.blocks = nn.ModuleList()

        # Choose the computational block based on the block_type
        for _ in range(depth):
            if block_type == "CSP":
                self.blocks.append(CSPBlock(in_channels, out_channels))
            elif block_type == "Res":
                self.blocks.append(ResidualBlock(in_channels, out_channels))
            elif block_type == "Conv":
                self.blocks.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=1, padding=1))
            else:
                raise ValueError(f"Unsupported block_type: {block_type}")

    def forward(self, x):
        """
        Aggregates features across layers with gradient optimization.
        """
        outputs = [x]  # Collect initial input
        for block in self.blocks:
            x = block(x)
            outputs.append(x)  # Aggregate outputs
        return torch.cat(outputs, dim=1)  # Channel-wise concatenation


class CSPBlock(nn.Module):
    """
    Example CSP block implementation.
    """
    def __init__(self, in_channels, out_channels):
        super(CSPBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels // 2, kernel_size=1, stride=1, bias=False)
        self.conv2 = nn.Conv2d(out_channels // 2, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn = nn.BatchNorm2d(out_channels)
        self.activation = nn.SiLU()  # Swish activation

    def forward(self, x):
        y = self.conv1(x)
        y = self.conv2(y)
        return self.activation(self.bn(y))


class ResidualBlock(nn.Module):
    """
    Example residual block implementation.
    """
    def __init__(self, in_channels, out_channels):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.activation = nn.ReLU(inplace=True)

    def forward(self, x):
        identity = x
        out = self.conv1(x)
        out = self.bn1(out)
        return self.activation(out + identity)


class GELAN(nn.Module):
    """
    Full GELAN module with configurable blocks and depths.
    """
    def __init__(self, in_channels, out_channels, num_blocks=3, block_type="CSP", depth=2):
        super(GELAN, self).__init__()
        self.gelan_blocks = nn.ModuleList([
            GELANBlock(in_channels, out_channels, block_type, depth) for _ in range(num_blocks)
        ])
        self.final_conv = nn.Conv2d(out_channels * num_blocks, out_channels, kernel_size=1)

    def forward(self, x):
        for gelan_block in self.gelan_blocks:
            x = gelan_block(x)
        return self.final_conv(x)


# Example usage
if __name__ == "__main__":
    input_tensor = torch.randn(1, 64, 128, 128)  # Batch size 1, 64 channels, 128x128 resolution
    model = GELAN(in_channels=64, out_channels=128, num_blocks=2, block_type="CSP", depth=3)
    output = model(input_tensor)
    print(f"Output shape: {output.shape}")



RuntimeError: Given groups=1, weight of size [64, 64, 1, 1], expected input[1, 128, 128, 128] to have 64 channels, but got 128 channels instead

Ram Sam


In [28]:
import torch
import torch.nn as nn


class CustomBlock(nn.Module):
    """
    A block that processes a series of convolutions and a concatenation step.
    """
    def __init__(self):
        super(CustomBlock, self).__init__()
        
        # Define convolutional layers
        self.conv1 = nn.Conv2d(256, 256, kernel_size=1, stride=1, bias=False)
        self.conv2 = nn.Conv2d(256, 256, kernel_size=1, stride=1, bias=False)
        self.conv3 = nn.Conv2d(256, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv4 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv5 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.conv6 = nn.Conv2d(128, 128, kernel_size=3, stride=1, padding=1, bias=False)
        self.final_conv = nn.Conv2d(1024, 256, kernel_size=1, stride=1, bias=False)
        
        # Define batch normalization and activation
        self.bn = nn.BatchNorm2d(256)
        self.activation = nn.SiLU()  # Swish activation

    def forward(self, x):
        # First two convolutional layers
        x1 = self.conv1(x)
        x2 = self.conv2(x)
        
        # Series of 3x3 convolutions
        x3 = self.conv3(x2)
        x4 = self.conv4(x3)
        x5 = self.conv5(x4)
        x6 = self.conv6(x5)
        print(x1.shape, x2.shape, x3.shape, x4.shape, x5.shape, x6.shape,)
        
        # Concatenate outputs
        concatenated = torch.cat([x1, x2, x3, x4, x5, x6], dim=1)
        print(concatenated.shape)
        
        # Final 1x1 convolution
        out = self.final_conv(concatenated)
        out = self.bn(out)
        return self.activation(out)


input_tensor = torch.randn(1, 256, 64, 64)  # Batch size 1, 256 channels, 64x64 resolution
model = CustomBlock()
output = model(input_tensor)
print(f"Output shape: {output.shape}")


torch.Size([1, 256, 64, 64]) torch.Size([1, 256, 64, 64]) torch.Size([1, 128, 64, 64]) torch.Size([1, 128, 64, 64]) torch.Size([1, 128, 64, 64]) torch.Size([1, 128, 64, 64])
torch.Size([1, 1024, 64, 64])
Output shape: torch.Size([1, 256, 64, 64])


In [27]:
torch.cat([input_tensor, input_tensor], dim=1).shape

torch.Size([1, 512, 64, 64])

In [29]:
import torch
import torch.nn as nn

class Network(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.module_list = nn.ModuleList()
        self.cfg = cfg

        for i, (from_layers, num, module, args) in enumerate(cfg):
            if module == 'Conv':
                out_channels, kernel_size, stride = args
                padding = (kernel_size - 1) // 2
                self.module_list.append(nn.Conv2d(in_channels=0, out_channels=out_channels, kernel_size=kernel_size, stride=stride, padding=padding)) #in_channels will be determined later

            elif module == 'Concat':
                self.module_list.append(nn.Identity()) # Placeholder for concat

            elif module == 'MP':
                self.module_list.append(nn.MaxPool2d(kernel_size=2, stride=2))
            else:
                raise ValueError(f"Unknown module type: {module}")

    def forward(self, x):
        outputs = []
        for i, (from_layers, num, module, args) in enumerate(self.cfg):
            if isinstance(from_layers, int):
                if from_layers == -1:
                    x = self.module_list[i](x)
                else:
                    x = self.module_list[i](outputs[from_layers])
            elif isinstance(from_layers, list):
                concatenated_inputs = []
                for from_layer in from_layers:
                    concatenated_inputs.append(outputs[from_layer])
                x = torch.cat(concatenated_inputs, dim=1)
                x = self.module_list[i](x) # apply the conv after concat
            outputs.append(x)
        return x

# Example usage:
cfg = [[-1, 1, 'Conv', [32, 3, 1]],
       [-1, 1, 'Conv', [64, 3, 2]],
       [-1, 1, 'Conv', [64, 3, 1]],
       [-1, 1, 'Conv', [128, 3, 2]],
       [-1, 1, 'Conv', [64, 1, 1]],
       [-2, 1, 'Conv', [64, 1, 1]],
       [-1, 1, 'Conv', [64, 3, 1]],
       [-1, 1, 'Conv', [64, 3, 1]],
       [-1, 1, 'Conv', [64, 3, 1]],
       [-1, 1, 'Conv', [64, 3, 1]],
       [[-1, -3, -5, -6], 1, 'Concat', [1]],
       [-1, 1, 'Conv', [256, 1, 1]],
       [-1, 1, 'MP', []],
       [-1, 1, 'Conv', [128, 1, 1]],
       [-3, 1, 'Conv', [128, 1, 1]],
       [-1, 1, 'Conv', [128, 3, 2]],
       [[-1, -3], 1, 'Concat', [1]],
       [-1, 1, 'Conv', [128, 1, 1]],
       [-2, 1, 'Conv', [128, 1, 1]],
       [-1, 1, 'Conv', [128, 3, 1]],
       [-1, 1, 'Conv', [128, 3, 1]],
       [-1, 1, 'Conv', [128, 3, 1]],
       [-1, 1, 'Conv', [128, 3, 1]],
       [[-1, -3, -5, -6], 1, 'Concat', [1]],
       [-1, 1, 'Conv', [512, 1, 1]],
       [-1, 1, 'MP', []],
       [-1, 1, 'Conv', [256, 1, 1]],
       [-3, 1, 'Conv', [256, 1, 1]],
       [-1, 1, 'Conv', [256, 3, 2]],
       [[-1, -3], 1, 'Concat', [1]],
       [-1, 1, 'Conv', [256, 1, 1]],
       [-2, 1, 'Conv', [256, 1, 1]],
       [-1, 1, 'Conv', [256, 3, 1]],
       [-1, 1, 'Conv', [256, 3, 1]],
       [-1, 1, 'Conv', [256, 3, 1]],
       [-1, 1, 'Conv', [256, 3, 1]],
       [[-1, -3, -5, -6], 1, 'Concat', [1]],
       [-1, 1, 'Conv', [1024, 1, 1]],
       [-1, 1, 'MP', []],
       [-1, 1, 'Conv', [512, 1, 1]],
       [-3, 1, 'Conv', [512, 1, 1]],
       [-1, 1, 'Conv', [512, 3, 2]],
       [[-1, -3], 1, 'Concat', [1]],
       [-1, 1, 'Conv', [256, 1, 1]],
       [-2, 1, 'Conv', [256, 1, 1]],
       [-1, 1, 'Conv', [256, 3, 1]],
       [-1, 1, 'Conv', [256, 3, 1]],
       [-1, 1, 'Conv', [256, 3, 1]],
       [-1, 1, 'Conv', [256, 3, 1]],
       [[-1, -3, -5, -6], 1, 'Concat', [1]],
       [-1, 1, 'Conv', [1024, 1, 1]]
       ]

model = Network(cfg)
test_input = torch.randn(1, 3, 224, 224) # Example input
output = model(test_input)
print(output.shape)

C:\Users\Nihar\anaconda3\envs\tf\lib\site-packages\torch\nn\init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


RuntimeError: Given groups=1, weight of size [32, 0, 3, 3], expected input[1, 3, 224, 224] to have 0 channels, but got 3 channels instead